In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').getOrCreate()

23/09/18 08:14:54 WARN Utils: Your hostname, elvin-Aspire-E1-571 resolves to a loopback address: 127.0.1.1; using 192.168.8.183 instead (on interface wlp3s0)
23/09/18 08:14:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/18 08:14:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Preparando Dados para Regressão

In [2]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [3]:
carros_temp = spark.read.csv("/home/elvin/download/Carros.csv", inferSchema=True, header=True, sep=";")
carros_temp.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
|    181|        6|        225|            276| 346| 2022|        1|          0|      3|          1|105|
|    143|        8|        360|            321| 357| 15

In [4]:
carros = carros_temp.select("Consumo", "Cilindros", "Cilindradas", "HP")
carros.show()

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
|    181|        6|        225|105|
|    143|        8|        360|245|
|    244|        4|       1467| 62|
|    228|        4|       1408| 95|
|    192|        6|       1676|123|
|    178|        6|       1676|123|
|    164|        8|       2758|180|
|    173|        8|       2758|180|
|    152|        8|       2758|180|
|    104|        8|        472|205|
|    104|        8|        460|215|
|    147|        8|        440|230|
|    324|        4|        787| 66|
|    304|        4|        757| 52|
|    339|        4|        711| 65|
+-------+---------+-----------+---+
only showing top 20 rows



**Adicionando o vetor características**

In [5]:
vectCaracterísticas = VectorAssembler(inputCols=[('Consumo'), ('Cilindros'), ('Cilindradas')], outputCol='caracteristicas')
carros = vectCaracterísticas.transform(carros)
carros.show()

+-------+---------+-----------+---+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|
+-------+---------+-----------+---+------------------+
|     21|        6|        160|110|  [21.0,6.0,160.0]|
|     21|        6|        160|110|  [21.0,6.0,160.0]|
|    228|        4|        108| 93| [228.0,4.0,108.0]|
|    214|        6|        258|110| [214.0,6.0,258.0]|
|    187|        8|        360|175| [187.0,8.0,360.0]|
|    181|        6|        225|105| [181.0,6.0,225.0]|
|    143|        8|        360|245| [143.0,8.0,360.0]|
|    244|        4|       1467| 62|[244.0,4.0,1467.0]|
|    228|        4|       1408| 95|[228.0,4.0,1408.0]|
|    192|        6|       1676|123|[192.0,6.0,1676.0]|
|    178|        6|       1676|123|[178.0,6.0,1676.0]|
|    164|        8|       2758|180|[164.0,8.0,2758.0]|
|    173|        8|       2758|180|[173.0,8.0,2758.0]|
|    152|        8|       2758|180|[152.0,8.0,2758.0]|
|    104|        8|        472|205| [104.0,8.0,472.0]|
|    104| 

**Separando em treino e teste**

In [6]:
carrosTreino, carrosTeste = carros.randomSplit([0.7,0.3])

In [7]:
print(carrosTreino.count())
print(carrosTeste.count())

20
12


# Criando um modelo de regressão linear

In [8]:
regLinear = LinearRegression(featuresCol="caracteristicas",labelCol="HP")
modelo = regLinear.fit(carrosTreino)

23/09/18 08:15:08 WARN Instrumentation: [c5c445b0] regParam is zero, which might cause numerical instability and overfitting.


In [10]:
previsao = modelo.transform(carrosTeste)

In [11]:
previsao.show()

+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|     21|        6|        160|110|  [21.0,6.0,160.0]|177.02973841139345|
|    104|        8|        460|215| [104.0,8.0,460.0]|214.79456622997222|
|    104|        8|        472|205| [104.0,8.0,472.0]|214.68961881104536|
|    133|        8|        350|245| [133.0,8.0,350.0]|210.11150888440972|
|    152|        8|       2758|180|[152.0,8.0,2758.0]|  185.353562508413|
|    178|        6|       1676|123|[178.0,6.0,1676.0]|133.21011114240514|
|    215|        4|       1201| 97|[215.0,4.0,1201.0]| 73.61683402385269|
|    244|        4|       1467| 62|[244.0,4.0,1467.0]| 65.64542421858147|
|    273|        4|         79| 66|  [273.0,4.0,79.0]| 72.13926698873104|
|    304|        4|        757| 52| [304.0,4.0,757.0]| 60.17534692542622|
|    304|        4|        951|113| [3

In [12]:
avaliar = RegressionEvaluator(predictionCol="prediction", labelCol="HP", metricName="rmse")

In [13]:
rmse = avaliar.evaluate(previsao)

In [14]:
print(rmse)

28.392774083634993


**Criando um modelo de Random Forest Regression**

In [15]:
rfreg = RandomForestRegressor(featuresCol="caracteristicas", labelCol="HP")

In [16]:
modelo2 = rfreg.fit(carrosTreino)

23/09/18 08:20:46 WARN DecisionTreeMetadata: DecisionTree reducing maxBins from 32 to 20 (= number of training instances)


In [17]:
previsao2 = modelo2.transform(carrosTeste)

In [18]:
previsao2.show()

+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|     21|        6|        160|110|  [21.0,6.0,160.0]|116.85416666666666|
|    104|        8|        460|215| [104.0,8.0,460.0]| 224.7443055555556|
|    104|        8|        472|205| [104.0,8.0,472.0]| 224.7443055555556|
|    133|        8|        350|245| [133.0,8.0,350.0]| 227.7693055555556|
|    152|        8|       2758|180|[152.0,8.0,2758.0]| 214.6443055555556|
|    178|        6|       1676|123|[178.0,6.0,1676.0]|125.72583333333333|
|    215|        4|       1201| 97|[215.0,4.0,1201.0]| 94.60869047619047|
|    244|        4|       1467| 62|[244.0,4.0,1467.0]| 90.19619047619048|
|    273|        4|         79| 66|  [273.0,4.0,79.0]|104.56285714285714|
|    304|        4|        757| 52| [304.0,4.0,757.0]|             75.06|
|    304|        4|        951|113| [3

In [19]:
previsao.show()

+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|     21|        6|        160|110|  [21.0,6.0,160.0]|177.02973841139345|
|    104|        8|        460|215| [104.0,8.0,460.0]|214.79456622997222|
|    104|        8|        472|205| [104.0,8.0,472.0]|214.68961881104536|
|    133|        8|        350|245| [133.0,8.0,350.0]|210.11150888440972|
|    152|        8|       2758|180|[152.0,8.0,2758.0]|  185.353562508413|
|    178|        6|       1676|123|[178.0,6.0,1676.0]|133.21011114240514|
|    215|        4|       1201| 97|[215.0,4.0,1201.0]| 73.61683402385269|
|    244|        4|       1467| 62|[244.0,4.0,1467.0]| 65.64542421858147|
|    273|        4|         79| 66|  [273.0,4.0,79.0]| 72.13926698873104|
|    304|        4|        757| 52| [304.0,4.0,757.0]| 60.17534692542622|
|    304|        4|        951|113| [3

In [20]:
rmse2 = avaliar.evaluate(previsao2)
print(rmse2)

23.053293092274842


In [21]:
print(rmse)

28.392774083634993
